### Importing packages we will need.

In [563]:
from ortools.linear_solver import pywraplp
from numpy import average, std
import json
import pandas as pd
import requests
import math
import numpy as np
import warnings
warnings.filterwarnings('ignore')

### Reading all the data from the gitlab we will use in the next steps.

In [564]:
response1 = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response1.text)
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/estaciones.json")
stations_db = json.loads(response.text)
with open("./functions/neighbors_with_9_minute_station.json", "r") as f:
    neighbors_with_9_minute_station = json.load(f)
s = pd.read_csv("./aux_data/skl.csv",index_col=0)
d = pd.read_csv("./aux_data/dlk.csv",index_col=0)
b = pd.read_csv("./aux_data/bijk.csv",index_col=0)
Pkv = pd.read_csv("./aux_data/pkv.csv", index_col=0)
Skl =pd.read_csv("aux_data/skl.csv")

### Initial information imposed by the problem.

In [565]:
total_vehicles = {
    "ha": np.float64(1),
    "engines": np.float64(197),
    "ladders": np.float64(143),
    "rescue": np.float64(5),
    "squads": np.float64(8),

}

In [566]:
neighbourhoods = list(neighbors_with_9_minute_station.keys())
K = len(neighbourhoods)
shifts = ["9 AM to 6 PM", "6 PM until 9 AM"]
I = len(shifts)
stations = list(distances_stations_ntas_db.keys())
L = len(stations)
vehicles = ["engines", "squads", "ladders", "ha", "rescue"]
J = len(vehicles)

In [567]:
Cl = {}
for st in stations_db:
    Cl[stations.index(st["address"])] = st["capacity"]

### Auxiliar functions to work with indices instead of using categorical variables.

In [568]:
def index2shift(i):
    return shifts[i]

def shift2index(shift):
    return shifts.index(shift)

def index2vehicle(j):
    return vehicles[j]

def vehicle2index(vehicle):
    return vehicles.index(vehicle)

def index2neighbourhood(k):
    return neighbourhoods[k]

def neighbourhood2index(neighbourhood):
    return neighbourhoods.index(neighbourhood)

def index2station(l):
    return stations[l]

def station2index(station):
    return stations.index(station)
    

Expand b matrix in order to have a row for each neighbor. We will need the matrix with a length of 390 in order to avoid future errors.

In [569]:
first = b[b['shift']==1]
second = b[b['shift']==0]

In [570]:
aux = first['nta']
for elem in neighbourhoods:
    if elem not in aux:
        df = pd.Series(data={'nta':elem,'engines':1,'squads':1,'ladders':1,'ha':1,'rescue':1,'shift':1})
        first.loc[len(first)]=df

first = first.drop_duplicates(subset=['nta']).reset_index().drop(['index'],axis=1)

In [571]:
aux1 = second['nta']
for elem in neighbourhoods:
    if elem not in aux1:
        df = pd.Series(data={'nta':elem,'engines':1,'squads':1,'ladders':1,'ha':1,'rescue':1,'shift':0})
        second.loc[len(second)]=df

second = second.drop_duplicates(subset=['nta']).reset_index().drop(['index'],axis=1)

aux1 = second['nta']
for elem in neighbourhoods:
    if elem not in aux1:
        df = pd.Series(data={'nta':elem,'engines':1,'squads':1,'ladders':1,'ha':1,'rescue':1,'shift':0})
        second.loc[len(second)]=df


second = second.drop_duplicates(subset=['nta']).reset_index().drop(['index'],axis=1)

second = second.drop_duplicates(subset=['nta']).reset_index().drop(['index'],axis=1)
      

In [572]:
b= pd.concat([first,second],axis=0).reset_index().drop(["index"],axis=1)

#### Reading the matrix that gives us the information to be aware if the distance from station l to neighbor k is less or equal than 540s (9 minutes).

In [573]:
# Load reacheable matrix (binary), indicating if a station (row) reaches a neighbourhood center (column) in
# less than 9 minutes
s.index = pd.Series(list(s.index)).apply(lambda x: station2index(x))
s.columns = pd.Series(list(s.columns)).apply(lambda x: neighbourhood2index(x))

#### Reading the matrix that contains the information about the distances between stations and neighbors.

In [574]:
# Load distance matrix from station (row) to neighbourhood center (column)

d.index = pd.Series(list(d.index)).apply(lambda x: station2index(x))
d.columns = pd.Series(list(d.columns)).apply(lambda x: neighbourhood2index(x))

If we apply the product of reacheable matrices and distance matrix we get a matrix where adding the distances per row will serve as a cost for setting certain types of vehicles in that station.

This happens because our model indicates that when setting a hazmat, rescue or squat vehicle in a station, it will serve all the neighbourhoods which center is reacheable in less than 9 minutes.

In [575]:
# The cost of assigning a vehicle (hazmat, rescue or squad) to a station in a certain shift will result from 
#the weighted combination of the distances of the reachable neighbourhoods depending on the estimated demand
cost_matrix = d*s
cost = {}
for i in range(len(b)):
      cost[(neighbourhood2index(b.iloc[i]["nta"]), "ha", b.iloc[i]["shift"])] = b.iloc[i]["ha"]
      cost[(neighbourhood2index(b.iloc[i]["nta"]), "squads", b.iloc[i]["shift"])] = b.iloc[i]["squads"]
      cost[(neighbourhood2index(b.iloc[i]["nta"]), "rescue", b.iloc[i]["shift"])] = b.iloc[i]["rescue"]

costs = {}
for i in range(len(cost_matrix)):
      costs[("ha", 0)] = costs.get(("ha", 0), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "ha", 0), 0)) for j in range(cost_matrix.shape[1]))]
      costs[("ha", 1)] = costs.get(("ha", 1), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "ha", 1), 0)) for j in range(cost_matrix.shape[1]))]
      costs[("squads", 0)] = costs.get(("squads", 0), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "squads", 0), 0)) for j in range(cost_matrix.shape[1]))]
      costs[("squads", 1)] = costs.get(("squads", 1), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "squads", 1), 0)) for j in range(cost_matrix.shape[1]))]
      costs[("rescue", 0)] = costs.get(("rescue", 0), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "rescue", 0), 0)) for j in range(cost_matrix.shape[1]))]
      costs[("rescue", 1)] = costs.get(("rescue", 1), []) + [sum(cost_matrix.iloc[i][j]*(cost.get((j, "rescue", 1), 0)) for j in range(cost_matrix.shape[1]))]

cost_matrix["costhashift0"] = costs[("ha", 0)]
cost_matrix["costhashift1"] = costs[("ha", 1)]
cost_matrix["costsquadsshift0"] = costs[("squads", 0)]
cost_matrix["costsquadsshift1"] = costs[("squads", 1)]
cost_matrix["costrescueshift0"] = costs[("rescue", 0)]
cost_matrix["costrescueshift1"] = costs[("rescue", 1)]
cost_matrix["reacheable_neighbourhoods"] = s.apply(lambda x:sum(x),axis=1)  #total reacheable centers
cost_matrix["normalized_costhashift0"] = cost_matrix.apply(lambda x: x["costhashift0"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costhashift1"] = cost_matrix.apply(lambda x: x["costhashift1"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costsquadsshift0"] = cost_matrix.apply(lambda x: x["costsquadsshift0"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costsquadsshift1"] = cost_matrix.apply(lambda x: x["costsquadsshift1"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costrescueshift0"] = cost_matrix.apply(lambda x: x["costrescueshift0"]/x["reacheable_neighbourhoods"],axis=1)
cost_matrix["normalized_costrescueshift1"] = cost_matrix.apply(lambda x: x["costrescueshift1"]/x["reacheable_neighbourhoods"],axis=1)


In [576]:
cost_matrix["normalized_costhashift0"] = cost_matrix["normalized_costhashift0"].apply(lambda x: (x-min(cost_matrix["normalized_costhashift0"]))/(max(cost_matrix["normalized_costhashift0"])- min(cost_matrix["normalized_costhashift0"])))
cost_matrix["normalized_costhashift1"] = cost_matrix["normalized_costhashift1"].apply(lambda x: (x-min(cost_matrix["normalized_costhashift1"]))/(max(cost_matrix["normalized_costhashift1"])- min(cost_matrix["normalized_costhashift1"])))
cost_matrix["normalized_costsquadsshift0"] = cost_matrix["normalized_costsquadsshift0"].apply(lambda x: (x-min(cost_matrix["normalized_costsquadsshift0"]))/(max(cost_matrix["normalized_costsquadsshift0"])- min(cost_matrix["normalized_costsquadsshift0"])))
cost_matrix["normalized_costsquadsshift1"] = cost_matrix["normalized_costsquadsshift1"].apply(lambda x: (x-min(cost_matrix["normalized_costsquadsshift1"]))/(max(cost_matrix["normalized_costsquadsshift1"])- min(cost_matrix["normalized_costsquadsshift1"])))
cost_matrix["normalized_costrescueshift0"] = cost_matrix["normalized_costrescueshift0"].apply(lambda x: (x-min(cost_matrix["normalized_costrescueshift0"]))/(max(cost_matrix["normalized_costrescueshift0"])- min(cost_matrix["normalized_costrescueshift0"])))
cost_matrix["normalized_costrescueshift1"] = cost_matrix["normalized_costrescueshift1"].apply(lambda x: (x-min(cost_matrix["normalized_costrescueshift1"]))/(max(cost_matrix["normalized_costrescueshift1"])- min(cost_matrix["normalized_costrescueshift1"])))
cost_matrix=cost_matrix.fillna(2**8)

Parameter needed to make the 9 minute constraint work.

In [577]:
new_param=[]
cols = cost_matrix.columns[0:195]
for c in cols:
    suma = 0
    for e in cost_matrix[c]:
        suma+=e

    if suma >0:new_param.append(1)
    else:new_param.append(0)

Auxiliar factor dictionary.

In [578]:
factor_dict = {}
for v in ["ha","squads","rescue"]:
    for i in [0,1]:
        for n in list(b["nta"].unique()):
            factor_dict[(i,neighbourhood2index(n),v)] = b[v].loc[(b["nta"]==n)&(b["shift"]==i)].values
            

#### Define a function for simpler access to costs.

In [579]:
def get_costs(l, vehicle, shift):
    return cost_matrix["normalized_cost" + vehicle + f"shift{shift}"][l]

#### Build a matrix to see how many stations are reachable in 540s from a neighbor.

In [580]:
Bijk = b
newcols = []
for col in Bijk.columns[1:-1]:
    newcols.append(vehicle2index(col))
Bijk.columns = ["nta"] + newcols + ["shift"]
Bijk["nta"] = Bijk["nta"].apply(lambda x: neighbourhood2index(x))
Bijk = Bijk.apply(lambda x: x.apply(lambda y: math.ceil(y)))


#### Matrix that shows us which stations are 9 minutes away from an specific neighbor.

In [581]:
newcols = []
for col in Skl.columns[1:]:
    newcols.append(neighbourhood2index(col))
Skl.columns = ["station"]+newcols
Skl["station"] = Skl["station"].apply(lambda x: station2index(x))
Skl.head()

,station,0,1,2,3,4,5,6,7,8,...,185,186,187,188,189,190,191,192,193,194
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


#### Matrix that shows us which neighbors are contained in each borough.

In [582]:
boroughs = list(Pkv["index"].unique())

def index2boro(l):
    return boroughs[l]

def boro2index(station):
    return boroughs.index(station)

Pkv["index"] = Pkv["index"].apply(lambda x: boro2index(x))
Pkv.columns = np.hstack((np.array("Borough"),pd.Series(list(Pkv.columns[1:])).apply(lambda x: neighbourhood2index(x)).values))
B = len(Pkv["Borough"].unique())
print(B)
Pkv.head()

5


,Borough,0,1,2,3,4,5,6,7,8,...,185,186,187,188,189,190,191,192,193,194
0,0,1,0,1,0,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2,2,0,0,0,1,0,0,1,0,0,...,0,0,0,1,1,1,1,0,1,1
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Objective Function Definition

To recap, our objective function has the following aspect:

$$
Minimize \hspace{0.05cm} Z:
\sum_{i}^{I} \sum_{j \in \left\lbrace engine,ladder\right\rbrace} \sum_{k}^{K} \left( B_{i,j,k} - \sum_{l}^{L} x_{i,j,k,l} \right) + \sum_{i}^{I}\sum_{l}^{L}Ch_{l,i}*Yh_{l,i} + \sum_{i}^{I}\sum_{l}^{L}Cs_{l,i}*Ys_{l,i} + \sum_{i}^{I}\sum_{l}^{L}Cr_{l,i}*Yr_{l,i} + \sum_{l1,l2 \in \left\lbrace L\right\rbrace} N_{d->n,l1,l2} + \sum_{l1,l2 \in \left\lbrace L\right\rbrace} N_{n->d,l1,l2}
$$

In [583]:
solver_scip = pywraplp.Solver.CreateSolver("SCIP_MIXED_INTEGER_PROGRAMMING") 
solver_cbc = pywraplp.Solver.CreateSolver("CBC") 

In [584]:
solver = solver_scip
solver.SetTimeLimit(1000*60*10)


variables = {}


for i in range(I):
  for j in range(J):
    if j not in {1, 3, 4}: # If the vehicle is not of squad, hazardous or rescue type
      for k in range(K):
        for l in range(L):
          # Shifted to integer variable instead of real
          v = solver.IntVar(0, solver.infinity(), "Vehicles of type " + index2vehicle(j) + " assigned to station " + index2station(l) + " to cover the neighbourhood " + index2neighbourhood(k) + " in shift " + index2shift(i))
          variables[(i,j,k,l)] = v
    else:
      for l in range(L):
        v = solver.BoolVar("Vehicles of type " + index2vehicle(j) + " assigned to station " + index2station(l) + " in shift " + index2shift(i))
        variables[("Y",i,j,l)] = v

for l1 in range(L):
  for l2 in range(L):
    if l1!=l2:
      v = solver.IntVar(0, solver.infinity(), "Vehicles moved from station station " + index2station(l1) + " to station " + index2station(l2) + " in the change of shift (day->night)")
      variables[("N",0,l1,l2)] = v
      v = solver.IntVar(0, solver.infinity(), "Vehicles moved from station station " + index2station(l1) + " to station " + index2station(l2) + " in the change of shift (night->day)")
      variables[("N",1,l1,l2)] = v

for i in range(I):
  for l in range(L):
    v = solver.BoolVar("There is more than one vehicle in the station " + index2station(l) + " during shift " + index2shift(i))
    variables[(i,l)] = v

We will add parts to the objective function sequentially

### Add First Sumation

$$
\sum_{i}^{I} \sum_{j \in \left\lbrace engine,ladder\right\rbrace} \sum_{k}^{K} \left( B_{i,j,k} - \sum_{l}^{L} x_{i,j,k,l} \right) 
$$

ADD NORMALIZATION TERM IN THE EXPRESSION (`max_sc`)

In [585]:
max_sc = Bijk[[0,2]].sum().sum()

In [586]:
# 'combinations' dictionary will take account of those combinations of 
# neighbourhoods and shifts where there are no expected needs of vehicles of whatever type
combinations = {}
beta = 0
objective = solver.Objective()
objective.SetMinimization()
for i in range(I):
  for j in range(J):
    if j not in (1,3,4): #Hazmat,squat and rescue units are not included in this summation
      for k in range(K):
        try:
          # The neighbourhood k has needs of vehicles of type j in shift i
          beta += int(Bijk[(Bijk["nta"] == k) & (Bijk["shift"] == i)][j].item())
          combinations[i,j,k] = "Available"
          for l in range(L):
            v = variables[(i,j,k,l)]
            objective.SetCoefficient(v,-1/max_sc)
        except:
          combinations[i,j,k] = "Not available"
objective.SetOffset(beta/max_sc)


### Second, Third and Fourth Sumation (Hazmat, Squad and Rescue Allocation Costs)

$$
\sum_{i}^{I}\sum_{l}^{L}Ch_{l,i}*Yh_{l,i} + \sum_{i}^{I}\sum_{l}^{L}Cs_{l,i}*Ys_{l,i} + \sum_{i}^{I}\sum_{l}^{L}Cr_{l,i}*Yr_{l,i}
$$

In [587]:
#Squad vehicle
for i in range(I):
    for l in range(L):
        variable = variables[("Y",i,1,l)]
        objective.SetCoefficient(variable,get_costs(l, "squads", i)/total_vehicles["squads"]) 
 
#Hazardous-Hazmat vehicle
for i in range(I):
    for l in range(L):
        variable = variables[("Y",i,3,l)]
        objective.SetCoefficient(variable,get_costs(l, "ha", i)/total_vehicles["ha"])
       
#Rescue vehicle
for i in range(I):
    for l in range(L):
        variable = variables[("Y",i,4,l)]
        objective.SetCoefficient(variable,get_costs(l, "rescue", i)/total_vehicles["rescue"])      


### Last Sumation

$$
\sum_{l1,l2 \in \left\lbrace L\right\rbrace} N_{d->n,l1,l2} + \sum_{l1,l2 \in \left\lbrace L\right\rbrace} N_{n->d,l1,l2}
$$

ADD NORMALIZATION TERM IN THE EXPRESSION (`max_sc`)

In [588]:
for l1 in range(L):
  for l2 in range(L):
    if l1!=l2:
      for i in range(I):
        variable = variables[("N",i,l1,l2)]
        objective.SetCoefficient(variable,1/max_sc)

## Constraints Definition

### Limits

[limit shift i, vehicle j, neighbouthood k]:$\hspace{0.1cm}\sum_{l}^{L} X_{i,j,k,l} \leq \beta_{i,j,k} \hspace{0.1cm}\forall \hspace{0.1cm}i=1...I,\hspace{0.1cm}j=1...J,\hspace{0.1cm}k=1...K$


In [589]:
limit = []

for i in range(I):
  for j in range(J):
    for k in range(K):
        bijk = int(Bijk[(Bijk["nta"] == k) & (Bijk["shift"] == i)][j])
        c = solver.Constraint(0, bijk)
        for l in range(L):
          if (i,j,k,l) in variables:
            v = variables[(i,j,k,l)]
            c.SetCoefficient(v, 1)
        limit.append(c)
      



### Hazmat Vehicle
[hazmat shift i]:$\hspace{0.1cm}\sum_{l}^{L} Yh_{l,i} = 1\hspace{0.1cm}\forall \hspace{0.1cm}i=1...I$

In [590]:
haz = []

for i in range(I):
  c = solver.Constraint(1, 1)
  for l in range(L):
    v = variables[("Y",i,3,l)]
    c.SetCoefficient(v, 1)
  haz.append(c)


### Squads Vehicle

[squads shift i]:$\hspace{0.1cm}\sum_{l}^{L} Ys_{l,i} = 8\hspace{0.1cm}\forall \hspace{0.1cm}i=1...I$

In [591]:
squads = []

for i in range(I):
  c = solver.Constraint(8, 8)
  for l in range(L):
    v = variables[("Y",i,1,l)]
    c.SetCoefficient(v, 1)
  squads.append(c)

### Rescue Vehicle

[squads shift i]:$\hspace{0.1cm}\sum_{l}^{L} Ys_{l,i} = 5\hspace{0.1cm}\forall \hspace{0.1cm}i=1...I$

In [592]:
rescue = []

for i in range(I):
  c = solver.Constraint(5, 5)
  for l in range(L):
    v = variables[("Y",i,4,l)]
    c.SetCoefficient(v, 1)
  rescue.append(c)

### Rest of Vehicles

[vehicle j]:$\hspace{0.1cm}\sum_{i}^{I}\sum_{k}^{K}\sum_{l}^{L} X_{i,j,k,l} \leq\hspace{0.1cm} T_{j}\hspace{0.1cm}\forall \hspace{0.1cm}j=1...J$

In [593]:
vehiclej = []

for j in range(J):
  if j not in {1,3,4}:
    for i in range(I):
      c = solver.Constraint(0, total_vehicles[index2vehicle(j)])
      for l in range(L):
        for k in range(K):
          v = variables[(i,j,k,l)]
          c.SetCoefficient(v, 1)
      vehiclej.append(c)

### Nine-minute arrival

[nine minutes]:$\hspace{0.1cm}\sum_{j}^{J}\sum_{l}^{L} X_{i,j,k,l}*S_{k,l} + \sum_{l}^{L}(Yh_{li} + Ys_{li} + Yr_{li})*S_{k,l}\geq\hspace{0.1cm} E_{k}\hspace{0.1cm}\forall \hspace{0.1cm}i=1...I \hspace{0.1cm}k=1...K$

In [594]:
nine_minutes = []

for k in range(K):
  p = new_param[k]
  for i in range(I):
    c = solver.Constraint(p, solver.infinity())
    for l in range(L):
      skl = s.iloc[l,k]
      for j in range(J):
        if j not in {1, 3, 4}:
          v = variables[(i,j,k,l)]
          c.SetCoefficient(v, skl)
        else:
          v = variables[("Y",i,j,l)]
          c.SetCoefficient(v, skl)

    nine_minutes.append(c)

### Capacities of stations

[capacity]:$\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{i,j,k,l} + Yh_{l,i} + Ys_{l,i} + Yr_{l,i}\leq\hspace{0.1cm} C_{l}\hspace{0.1cm}\forall \hspace{0.1cm}i=1...I \hspace{0.1cm}l=1...L$

In [595]:
capacity = []

for l in range(L):
  cl = Cl[l]
  for i in range(I):
    c = solver.Constraint(-solver.infinity(), cl)
    for j in range(J):
      if j not in {1,3,4}:
        for k in range(K):
          v = variables[(i,j,k,l)]
          c.SetCoefficient(v, 1)
      else:
        v = variables[("Y",i,j,l)]
        c.SetCoefficient(v, 1)
      capacity.append(c)

### Linking the variables of assignment (X) with the displacements (N)

[displacement]:$\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{2,j,k,l} + Yh_{l,2} + Ys_{l,2} + Yr_{l,2} =\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{1,j,k,l} + Yh_{l,1} + Ys_{l,1} + Yr_{l,1} + \sum_{l_{2}}^{L} (N_{d->n,l_{2}, l} - N_{d->n,l,l_{2}}) \hspace{0.1cm}\forall \hspace{0.1cm}l=1...L \hspace{0.1cm}$

In [596]:
displacement2night = []

for l in range(L):
    c = solver.Constraint(-solver.infinity(), 0)
    v = variables[("Y",0,1,l)]
    c.SetCoefficient(v, 1)
    v = variables[("Y",1,1,l)]
    c.SetCoefficient(v, -1)
    v = variables[("Y",1,3,l)]
    c.SetCoefficient(v, -1)
    v = variables[("Y",0,3,l)]
    c.SetCoefficient(v, 1)
    v = variables[("Y",1,4,l)]
    c.SetCoefficient(v, -1)
    v = variables[("Y",0,4,l)]
    c.SetCoefficient(v, 1)
    for l2 in range(L):
        if l!=l2:
          v = variables[("N",0,l,l2)]
          c.SetCoefficient(v, -1)
          v = variables[("N",0,l2,l)]
          c.SetCoefficient(v, 1)
    for j in range(J):
      if j not in {1,3,4}:
        for k in range(K):
          v = variables[(0,j,k,l)]
          c.SetCoefficient(v, 1)
          v = variables[(1,j,k,l)]
          c.SetCoefficient(v, -1)
    displacement2night.append(c)

[displacement]:$\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{1,j,k,l} + Yh_{l,1} + Ys_{l,1} + Yr_{l,1} =\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{2,j,k,l} + Yh_{l,2} + Ys_{l,2} + Yr_{l,2} + \sum_{l_{2}}^{L} (N_{n->d,l_{2}, l} - N_{n->d,l,l_{2}}) \hspace{0.1cm}\forall \hspace{0.1cm}l=1...L \hspace{0.1cm}$

In [597]:
displacement2day = []

for l in range(L):
    c = solver.Constraint(-solver.infinity(), 0)
    v = variables[("Y",0,1,l)]
    c.SetCoefficient(v, -1)
    v = variables[("Y",1,1,l)]
    c.SetCoefficient(v, 1)
    v = variables[("Y",1,3,l)]
    c.SetCoefficient(v, 1)
    v = variables[("Y",0,3,l)]
    c.SetCoefficient(v, -1)
    v = variables[("Y",1,4,l)]
    c.SetCoefficient(v, 1)
    v = variables[("Y",0,4,l)]
    c.SetCoefficient(v, -1)
    for l2 in range(L):
        if l!=l2:
          v = variables[("N",1,l,l2)]
          c.SetCoefficient(v, -1)
          v = variables[("N",1,l2,l)]
          c.SetCoefficient(v, 1)
    for j in range(J):
      if i not in {1,3,4}:
        for k in range(K):
          v = variables[(0,j,k,l)]
          c.SetCoefficient(v, -1)
          v = variables[(1,j,k,l)]
          c.SetCoefficient(v, 1)
    displacement2day.append(c)

### Ensure a fair distribution

[fair distribution]:$\hspace{0.1cm}\sum_{i}^{I}\sum_{j}^{J}\sum_{k}^{K}\sum_{l}^{L} X_{i,j,k,l}*P_{k,b} \leq\hspace{0.1cm}0.3*\sum_{i}^{I}\sum_{j}^{J}\sum_{k}^{K}\sum_{l}^{L} X_{i,j,k,l} \hspace{0.1cm}\forall \hspace{0.1cm}b=1...B \hspace{0.1cm}$

In [598]:
fair_distribution = []
dic = {}
for bo in range(B):
    
 
    c = solver.Constraint(-solver.infinity(), 0)
    for k in range(k):
      p = int(Pkv.iloc[bo, k+1])
      for j in range(J):
        if j not in {1,3,4}:
          for l in range(L):
            for i in range(I):
              v = variables[(i,j,k,l)]
              c.SetCoefficient(v, p)
              c.SetCoefficient(v, -0.3)


In [ ]:
fair_distribution2 = []

for bo in range(B):
    for i in range(I):
      for j in (1,4):
        c = solver.Constraint(1, solver.infinity())
        for k in range(K):
          p = float(Pkv.iloc[bo, k+1])
          if p:
            for l in range(L):
                skl = int(s.iloc[l,k])
                v = variables[("Y",i,j,l)]
                c.SetCoefficient(v, skl)
        fair_distribution2.append(c)

### Linking the binary variable with the number of vehicles in a station in a shift

[more than one]:$\hspace{0.1cm} 2Y_{i,l} \leq\hspace{0.1cm}\sum_{j}^{J}\sum_{k}^{K} X_{i,j,k,l} + Yh_{li} + Ys_{li} + Yr_{li} \leq\hspace{0.1cm} C_{l}Y_{i,l}+1 \hspace{0.1cm}\forall \hspace{0.1cm}i=1...I \hspace{0.1cm}l=1...L$

In [599]:
more_than_one = []

for i in range(I):
  for l in range(L):
    c = solver.Constraint(0, solver.infinity())
    v = variables[(i,l)]
    c.SetCoefficient(v, -2)
    if j not in {1,3,4}:
        for k in range(K):
          v = variables[(i,j,k,l)]
          c.SetCoefficient(v, 1)
    else:
        v = variables[("Y",i,j,l)]
        c.SetCoefficient(v, 1)
    more_than_one.append(c)

for i in range(I):
  cl = Cl[l]
  for i in range(I):
    c = solver.Constraint(-solver.infinity(), 1)
    v = variables[(i,l)]
    c.SetCoefficient(v, -cl)
    if j not in {1,3,4}:
        for k in range(K):
          v = variables[(i,j,k,l)]
          c.SetCoefficient(v, 1)
    else:
        v = variables[("Y",i,j,l)]
        c.SetCoefficient(v, 1)
    more_than_one.append(c)

### Diversity in each station

[not just one type]:$\hspace{0.1cm}\sum_{k}^{K} X_{i,j,k,l} \leq\hspace{0.1cm}\sum_{j_{2}}^{J}\sum_{k}^{K} X_{i,j_{2},k,l} + Yh_{li} + Ys_{li} + Yr_{li} - Y_{i,l} \hspace{0.1cm}\forall \hspace{0.1cm}i=1...I \hspace{0.1cm}j=1...J \hspace{0.1cm}l=1...L$

In [600]:
not_just_one_type = []

for i in range(I):
    for j in range(J):
      for l in range(L):
        c = solver.Constraint(-solver.infinity(), 0)
        v = variables[(i,l)]
        c.SetCoefficient(v, -1)
        for k in range(k):
            v = variables[(i,j,k,l)]
            c.SetCoefficient(v, 1)
            for j2 in range(J):
                if j2 not in {1,3,4}:
                  v = variables[(i,j2,k,l)]
                  c.SetCoefficient(v, -1)
                else:
                  v = variables[("Y",i,j2,l)]
                  c.SetCoefficient(v, -1)
    not_just_one_type.append(c)

In [601]:
result = solver.Solve()

if result == solver.ABNORMAL :
  print("Execution finished by an error")
elif result == solver.FEASIBLE :
  print("In the specified time limit the solver has found a feasible solution")
  for i in range(I):
    for j in range(J):
      if j not in {1, 3, 4}: # If the vehicle is not of squad, hazardous or rescue type
        for k in range(K):
          for l in range(L):
            v = variables[(i,j,k,l)]
            if v.SolutionValue()>0:
              print(v, v.solution_value())
      else:
        for l in range(L):
          v = variables[("Y",i,j,l)]
          if v.SolutionValue()>0:
            print(v, v.solution_value())

  for l1 in range(L):
    for l2 in range(L):
      if l1!=l2:
        v = variables[("N",0,l1,l2)]
        if v.SolutionValue()>0:
          print(v, v.solution_value())
        v = variables[("N",1,l1,l2)]
        if v.SolutionValue()>0:
          print(v, v.solution_value())
  print("The value for the objective function is", objective.Value())
elif result == solver.INFEASIBLE :
  print("There is no feasible solution for the problem")
elif result == solver.NOT_SOLVED :
  print("In the specified time limit the solver has not found any feasible solution")
elif result == solver.OPTIMAL :
  print("In the specified time limit the solver has found the optimal solution")
  for i in range(I):
    for j in range(J):
      if j not in {1, 3, 4}: # If the vehicle is not of squad, hazardous or rescue type
        for k in range(K):
          for l in range(L):
            v = variables[(i,j,k,l)]
            if v.SolutionValue()>0:
              print(v, v.solution_value())
              
      else:
        for l in range(L):
          v = variables[("Y",i,j,l)]
          if v.SolutionValue()>0:
            print(v, v.solution_value())

  for l1 in range(L):
    for l2 in range(L):
      if l1!=l2:
        v = variables[("N",0,l1,l2)]
        if v.SolutionValue()>0:
          print(v, v.solution_value())
        v = variables[("N",1,l1,l2)]
        if v.SolutionValue()>0:
          print(v, v.solution_value())

  for i in range(I):
    for l in range(L):
      v = variables[(i,l)]
      print(v, v.solution_value())
  print("The optimal value for the objective function is", objective.Value())
elif result == solver.UNBOUNDED :
  print("The solution is unbounded")
else :
  print("Unknown error code")

In the specified time limit the solver has found the optimal solution
Vehicles of type engines assigned to station 218-44 97th Avenue to cover the neighbourhood QN08 in shift 9 AM to 6 PM 1.0
Vehicles of type engines assigned to station 145-50 Springfield Blvd. to cover the neighbourhood QN08 in shift 9 AM to 6 PM 2.0
Vehicles of type engines assigned to station 1352 Saint John's Place to cover the neighbourhood BX28 in shift 9 AM to 6 PM 1.0
Vehicles of type engines assigned to station 205 Greenpoint Ave. to cover the neighbourhood QN55 in shift 9 AM to 6 PM 1.0
Vehicles of type engines assigned to station 153-11 Hillside Avenue to cover the neighbourhood QN55 in shift 9 AM to 6 PM 1.0
Vehicles of type engines assigned to station 1573 Castleton Avenue to cover the neighbourhood QN55 in shift 9 AM to 6 PM 2.0
Vehicles of type engines assigned to station 1560 Drumgoole Road West to cover the neighbourhood QN55 in shift 9 AM to 6 PM 1.0
Vehicles of type engines assigned to station 885 Ho

#### Creating all the data we will need to display the results obtained

In [65]:
first_map = pd.DataFrame()
n = []
s = []
st = []
ve = []
o = []
for i in range(I):
    for j in range(J):
        if j not in {1, 3, 4}: 
          for k in range(K):
            for l in range(L):
              v = variables[(i,j,k,l)]
              if v.SolutionValue()>0:
                n.append(index2neighbourhood(k))
                s.append(index2shift(i))
                st.append(index2station(l))
                ve.append(index2vehicle(j))
                o.append(v.solution_value())

      



first_map["shift"]=s
first_map["station"]=st
first_map["vehicle"]=ve
first_map["neighbor"]=n
first_map["value"]=o

            

In [64]:
#first_map.to_csv("aux_data/map_value_xijk.csv")

In [70]:
second = pd.DataFrame()

s = []
st = []
ve = []
o = []
for i in range(I):
    for j in range(J):
        if j in {1, 3, 4}: 
            for l in range(L):
              v = variables[("Y",i,j,l)]
              if v.SolutionValue()>0:
                s.append(index2shift(i))
                st.append(index2station(l))
                ve.append(index2vehicle(j))
                o.append(v.solution_value())

second["shift"]=s
second["station"]=st
second["vehicle"]=ve
second["value"]=o

In [71]:
#second.to_csv("aux_data/map_value_yijl.csv")

In [72]:
third = pd.DataFrame()
o = []
d = []
shift1 = []
shift2= []
values = []

for l1 in range(L):
    for l2 in range(L):
      if l1!=l2:
        v = variables[("N",0,l1,l2)]
        if v.SolutionValue()>0:
          values.append(v.solution_value())
          shift1.append(index2shift(0))
          shift2.append(index2shift(1))
          o.append(index2station(l1))
          d.append(index2station(l2))
        v = variables[("N",1,l1,l2)]
        if v.SolutionValue()>0:
          values.append(v.solution_value())
          shift1.append(index2shift(1))
          shift2.append(index2shift(0))
          o.append(index2station(l1))
          d.append(index2station(l2))

third["origin"]=o
third["destination"]=d
third["in_shift"]=shift1
third["to_shift"]=shift2
third['values']=values

In [74]:
#third.to_csv("aux_data/map_value_nl1l2.csv")